Import required libraries

In [1]:
"""
%pip install -U numpy
%pip install -U pandas
%pip install -U requests
%pip install -U bs4
%pip install -U selenium
%pip install -U matplotlib
%pip install -U seaborn
%pip install -U plotly
%pip install -U scikit-learn
%pip install -U python-dateutil
%pip install -U lxml
%pip install -U colorama
%pip install -U datetime
%pip install -U selenium
"""

'\n%pip install -U numpy\n%pip install -U pandas\n%pip install -U requests\n%pip install -U bs4\n%pip install -U selenium\n%pip install -U matplotlib\n%pip install -U seaborn\n%pip install -U plotly\n%pip install -U scikit-learn\n%pip install -U python-dateutil\n%pip install -U lxml\n%pip install -U colorama\n%pip install -U datetime\n%pip install -U selenium\n'

In [2]:
import sys
import os
import datetime
import shutil    as shu
from   colorama  import Fore             

start_time = datetime.datetime.now()
w, h       = shu.get_terminal_size()

def printSeparator():
    print(Fore.GREEN + '-' * w + Fore.WHITE)
    
def logStep(msg):
    l1 = len(msg)
    l2 = w - l1 - 27

    print(Fore.WHITE  + str(datetime.datetime.now()) +  " " + 
          Fore.YELLOW + msg + Fore.WHITE + "-" * l2  )
    sys.stdout.flush()
    
def printDFinfo(name,dfName):
    printSeparator()
    print('Name: ',name)
    printSeparator()
    print(dfName.info())
    printSeparator()
    print(f'Row Count :{Fore.RED}')
    print(dfName.count(),Fore.WHITE)
    printSeparator()
    print(dfName.head())
    printSeparator()

In [3]:
import warnings               as     warn
import numpy                  as     np
import pandas                 as     pd
import requests
import datetime
from   bs4                    import BeautifulSoup
from   dateutil.relativedelta import *
from   selenium               import webdriver
from   sklearn.ensemble       import RandomForestClassifier, RandomForestRegressor
from   sklearn.linear_model   import LinearRegression, LogisticRegression
from   sklearn.metrics        import classification_report, precision_score
from   sklearn.neural_network import MLPClassifier
from   sklearn.preprocessing  import StandardScaler
warn.filterwarnings("ignore", category=Warning)
warn.filterwarnings("ignore", category=DeprecationWarning)
warn.filterwarnings("ignore", category=FutureWarning)
warn.filterwarnings("ignore", category=UserWarning)

In [4]:
start_time00 = datetime.datetime.now()
logStep("ENVIRONMENT PREPARATION")
print(F"OS Platform      : {sys.platform}")
print(F"OS Name          : {os.name}")
print(F"OS HOME          : {os.environ.get('HOME')}")
print(F"OS uName         : {os.uname().sysname}")
print(F"OS NodeName      : {os.uname().nodename}")
print(F"OS Release       : {os.uname().release}")
print(F"OS Release Ver   : {os.uname().version}")
print(F"OS Machine       : {os.uname().machine}")
print(F"Process ID       : {os.getpid()}")
print(F"Parent Process   : {os.getppid()}")
print(F"OS User          : {os.getlogin()}")
print(F"OS User ID       : {os.getuid()}")
print(F"OS Group ID      : {os.getgid()}")
print(F"OS Effective ID  : {os.geteuid()}")
print(F"OS Effective GID : {os.getegid()}")
print(F"Current dir      : {os.getcwd()}")
print(F"Python version   : {sys.version}")
print(F"Version info     : {sys.version_info}")
print(F"Python API Ver   : {sys.api_version}")
print(f"Selenium Ver     : {webdriver.__version__}")
print(f"Pandas Ver       : {pd.__version__}")
print(f"Numpy Ver        : {np.__version__}")
logStep("DONE");
end_time            = datetime.datetime.now()
step00_elapsed_time = end_time - start_time00
logStep(F"ELAPSED TIME: {step00_elapsed_time} seconds")

2023-10-11 14:22:41.467665 ENVIRONMENT PREPARATION------------------------------
OS Platform      : darwin
OS Name          : posix
OS HOME          : /Users/eneas
OS uName         : Darwin
OS NodeName      : MBPJES14M2.local
OS Release       : 23.1.0
OS Release Ver   : Darwin Kernel Version 23.1.0: Tue Oct  3 22:40:26 PDT 2023; root:xnu-10002.41.8~15/RELEASE_ARM64_T6020
OS Machine       : arm64
Process ID       : 92530
Parent Process   : 92380
OS User          : root
OS User ID       : 501
OS Group ID      : 20
OS Effective ID  : 501
OS Effective GID : 20
Current dir      : /Users/eneas/Documents/GitHub/RB_Module_24_Project_04/Notebooks
Python version   : 3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]
Version info     : sys.version_info(major=3, minor=11, micro=5, releaselevel='final', serial=0)
Python API Ver   : 1013
Selenium Ver     : 4.12.0
Pandas Ver       : 2.0.3
Numpy Ver        : 1.24.3
2023-10-11 14:22:41.469751 DONE-------------------------------------------------
2023

Build required data structures - Race DF

In [5]:
start_time = datetime.datetime.now()
logStep("Build required data structures - Race DF")
races = {'season'   : [], 'round'  : [], 'circuit_id': [], 'lat'    : [],
        'long'      : [], 'country': [], 'date'      : [], 'url'    : []}
url   = 'https://ergast.com/api/f1/{}.json'
for year in list(range(2022,2023)):
    r    = requests.get(url.format(year))
    json = r.json()
    for item in json['MRData']['RaceTable']['Races']:
        try:                   races['season'].append(int(item['season']))
        except Exception as e: races['season'].append(None)
        try:                   races['round'].append(int(item['round']))
        except Exception as e: races['round'].append(None)
        try:                   races['circuit_id'].append(item['Circuit']['circuitId'])
        except Exception as e: races['circuit_id'].append(None)
        try:                   races['lat'].append(float(item['Circuit']['Location']['lat']))
        except Exception as e: races['lat'].append(None)
        try:                   races['long'].append(float(item['Circuit']['Location']['long']))
        except Exception as e: races['long'].append(None)
        try:                   races['country'].append(item['Circuit']['Location']['country'])
        except Exception as e: races['country'].append(None)
        try:                   races['date'].append(item['date'])
        except Exception as e: races['date'].append(None)
        try:                   races['url'].append(item['url'])
        except Exception as e: races['url'].append(None)
races = pd.DataFrame(races)
printDFinfo('races',races)
logStep("DONE");
end_time            = datetime.datetime.now()
step01_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step01_elapsed_time} seconds")

2023-10-11 14:23:01.226526 Build required data structures - Race DF-------------
--------------------------------------------------------------------------------
Name:  races
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      22 non-null     int64  
 1   round       22 non-null     int64  
 2   circuit_id  22 non-null     object 
 3   lat         22 non-null     float64
 4   long        22 non-null     float64
 5   country     22 non-null     object 
 6   date        22 non-null     object 
 7   url         22 non-null     object 
dtypes: float64(2), int64(2), object(4)
memory usage: 1.5+ KB
None
--------------------------------------------------------------------------------
Row Count :
season        22
round         22
circuit_id    22
lat           22
long

Build required data structures - Rounds DF

In [6]:
start_time = datetime.datetime.now()
logStep("Build required data structures - Rounds DF")
rounds = []
i = 0
for year in np.array(races.season.unique()):
    rounds.append([year, list(races[races.season == year]['round'])])
    print(rounds[i][0],rounds[i][1],len(rounds[i][1]))
    i = i + 1
logStep("DONE");
end_time            = datetime.datetime.now()
step02_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step02_elapsed_time} seconds")

2023-10-11 14:23:08.665220 Build required data structures - Rounds DF-----------
2022 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22] 22
2023-10-11 14:23:08.666939 DONE-------------------------------------------------
2023-10-11 14:23:08.667365 ELAPSED TIME: 0:00:00.002137 seconds-----------------


Build required data structures - Race Results DF

In [7]:
start_time = datetime.datetime.now()
logStep("Build required data structures - Results DF")
results = {'season'       : [], 'round'      : [], 'circuit_id'   : [], 'driver'     : [],
           'date_of_birth': [], 'nationality': [], 'constructor'  : [], 'grid'       : [],
           'time'         : [], 'status'     : [], 'points'       : [], 'podium'     : []}
url     = 'http://ergast.com/api/f1/{}/{}/results.json'
for n in list(range(len(rounds))):
    for i in rounds[n][1]:
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()
        for item in json['MRData']['RaceTable']['Races'][0]['Results']:
            try:                   results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
            except Exception as e: results['season'].append(None)
            try:                   results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
            except Exception as e: results['round'].append(None)
            try:                   results['circuit_id'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
            except Exception as e: results['circuit_id'].append(None)
            try:                   results['driver'].append(item['Driver']['driverId'])
            except Exception as e: results['driver'].append(None)
            try:                   results['date_of_birth'].append(item['Driver']['dateOfBirth'])
            except Exception as e: results['date_of_birth'].append(None)
            try:                   results['nationality'].append(item['Driver']['nationality'])
            except Exception as e: results['nationality'].append(None)
            try:                   results['constructor'].append(item['Constructor']['constructorId'])
            except Exception as e: results['constructor'].append(None)
            try:                   results['grid'].append(int(item['grid']))
            except Exception as e: results['grid'].append(None)
            try:                   results['time'].append(int(item['Time']['millis']))
            except Exception as e: results['time'].append(None)
            try:                   results['status'].append(item['status'])
            except Exception as e: results['status'].append(None)
            try:                   results['points'].append(int(item['points']))
            except Exception as e: results['points'].append(None)
            try:                   results['podium'].append(int(item['position']))
            except Exception as e: results['podium'].append(None)
results = pd.DataFrame(results)
printDFinfo('results',results)
logStep("DONE");
end_time            = datetime.datetime.now()
step03_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step03_elapsed_time} seconds")

2023-10-11 14:23:12.117835 Build required data structures - Results DF----------
--------------------------------------------------------------------------------
Name:  results
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   season         440 non-null    int64  
 1   round          440 non-null    int64  
 2   circuit_id     440 non-null    object 
 3   driver         440 non-null    object 
 4   date_of_birth  440 non-null    object 
 5   nationality    440 non-null    object 
 6   constructor    440 non-null    object 
 7   grid           440 non-null    int64  
 8   time           279 non-null    float64
 9   status         440 non-null    object 
 10  points         440 non-null    int64  
 11  podium         440 non-null    int64  
dtypes: float64(1), i

Build required data structures - Driver Standings DF

In [8]:
start_time = datetime.datetime.now()
logStep("Build required data structures - Driver Standings DF")
driver_standings = {'season'     : [], 'round'               : [], 
                    'driver'     : [], 'driver_points'       : [],
                    'driver_wins': [], 'driver_standings_pos': []}
url              = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'

def lookup (df, team, points):
    df['lookup1'] = df.season.astype(str) + df[team] + df['round'].astype(str)
    df['lookup2'] = df.season.astype(str) + df[team] + (df['round']-1).astype(str)
    new_df        = df.merge(df[['lookup1', points]], how = 'left', left_on='lookup2',right_on='lookup1')
    new_df.drop(['lookup1_x', 'lookup2', 'lookup1_y'], axis = 1, inplace = True)
    new_df.rename(columns={f'{points}_x': f'{points}_after_race', f'{points}_y': points}, inplace = True)
    new_df[points].fillna(0, inplace = True)
    return new_df

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()
        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            try:                   driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except Exception as e: driver_standings['season'].append(None)
            try:                   driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except Exception as e: driver_standings['round'].append(None)
            try:                   driver_standings['driver'].append(item['Driver']['driverId'])
            except Exception as e: driver_standings['driver'].append(None)
            try:                   driver_standings['driver_points'].append(int(item['points']))
            except Exception as e: driver_standings['driver_points'].append(None)
            try:                   driver_standings['driver_wins'].append(int(item['wins']))
            except Exception as e: driver_standings['driver_wins'].append(None)
            try:                   driver_standings['driver_standings_pos'].append(int(item['position']))
            except Exception as e: driver_standings['driver_standings_pos'].append(None)
driver_standings = pd.DataFrame(driver_standings)
driver_standings = lookup(driver_standings, 'driver', 'driver_points')
driver_standings = lookup(driver_standings, 'driver', 'driver_wins')
driver_standings = lookup(driver_standings, 'driver', 'driver_standings_pos')
driver_standings.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'], axis    = 1, inplace = True)
printDFinfo('driver_standings',driver_standings)
logStep("DONE");
end_time            = datetime.datetime.now()
step04_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step04_elapsed_time} seconds")

2023-10-11 14:23:26.156100 Build required data structures - Driver Standings DF-
--------------------------------------------------------------------------------
Name:  driver_standings
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   season                467 non-null    int64  
 1   round                 467 non-null    int64  
 2   driver                467 non-null    object 
 3   driver_points         467 non-null    float64
 4   driver_wins           467 non-null    float64
 5   driver_standings_pos  467 non-null    float64
dtypes: float64(3), int64(2), object(1)
memory usage: 22.0+ KB
None
--------------------------------------------------------------------------------
Row Count :
season                  467
round                   467
driv

Build required data structures - Constructor DF

In [9]:
start_time = datetime.datetime.now()
logStep("Build required data structures - Constructor DF")
constructor_rounds    = rounds
constructor_standings = {'season'          : [], 'round'                    : [],
                         'constructor'     : [], 'constructor_points'       : [],
                         'constructor_wins': [], 'constructor_standings_pos': []}
url                   = 'https://ergast.com/api/f1/{}/{}/constructorStandings.json'
for n in list(range(len(constructor_rounds))):
    for i in constructor_rounds[n][1]:
        r    = requests.get(url.format(constructor_rounds[n][0], i))
        json = r.json()
        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
            try:                   constructor_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            except Exception as e: constructor_standings['season'].append(None)
            try:                   constructor_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            except Exception as e: constructor_standings['round'].append(None)
            try:                   constructor_standings['constructor'].append(item['Constructor']['constructorId'])
            except Exception as e: constructor_standings['constructor'].append(None)
            try:                   constructor_standings['constructor_points'].append(int(item['points']))
            except Exception as e: constructor_standings['constructor_points'].append(None)
            try:                   constructor_standings['constructor_wins'].append(int(item['wins']))
            except Exception as e: constructor_standings['constructor_wins'].append(None)
            try:                   constructor_standings['constructor_standings_pos'].append(int(item['position']))
            except Exception as e: constructor_standings['constructor_standings_pos'].append(None)
constructor_standings = pd.DataFrame(constructor_standings)
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_points')
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_wins')
constructor_standings = lookup(constructor_standings, 'constructor', 'constructor_standings_pos')
constructor_standings.drop(['constructor_points_after_race','constructor_wins_after_race','constructor_standings_pos_after_race'],axis=1,inplace=True)
printDFinfo('Constructor_Standings',constructor_standings)
logStep("DONE");
end_time            = datetime.datetime.now()
step05_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step05_elapsed_time} seconds")

2023-10-11 14:24:50.117906 Build required data structures - Constructor DF------
--------------------------------------------------------------------------------
Name:  Constructor_Standings
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   season                     220 non-null    int64  
 1   round                      220 non-null    int64  
 2   constructor                220 non-null    object 
 3   constructor_points         220 non-null    float64
 4   constructor_wins           220 non-null    float64
 5   constructor_standings_pos  220 non-null    float64
dtypes: float64(3), int64(2), object(1)
memory usage: 10.4+ KB
None
--------------------------------------------------------------------------------
Row Count :
season         

Build required data structures - Qualifying DF

In [10]:
start_time = datetime.datetime.now()
logStep("Build required data structures - Qualifying DF")
qualifying_results = pd.DataFrame()
for year in list(range(2022,2023)):
    url = 'https://www.formula1.com/en/results.html/{}/races.html'
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, 'html.parser')
    year_links = []
    for page in soup.find_all('a', attrs = {'class':"resultsarchive-filter-item-link FilterTrigger"}):
        link = page.get('href')
        if f'/en/results.html/{year}/races/' in link: 
            year_links.append(link)
    year_df = pd.DataFrame()
    new_url = 'https://www.formula1.com{}'
    for n, link in list(enumerate(year_links)):
        link = link.replace('race-result.html', 'starting-grid.html')
        df = pd.read_html(new_url.format(link))
        df = df[0]
        df['season'] = year
        df['round'] = n+1
        for col in df:
            if 'Unnamed' in col:
                df.drop(col, axis = 1, inplace = True)
        year_df = pd.concat([year_df, df])
    qualifying_results = pd.concat([qualifying_results, year_df])
qualifying_results.rename(columns = {'Pos': 'grid', 'Driver': 'driver_name', 'Car': 'car', 'Time': 'qualifying_time'}, inplace = True)
qualifying_results.drop('No', axis = 1, inplace = True)
qualifying_results.qualifying_time = 1 #qualifying_results.grid
printDFinfo('Qualifying_Results',qualifying_results)
logStep("DONE");
end_time            = datetime.datetime.now()
step06_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step06_elapsed_time} seconds")

2023-10-11 14:25:14.141629 Build required data structures - Qualifying DF-------
--------------------------------------------------------------------------------
Name:  Qualifying_Results
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 436 entries, 0 to 19
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   grid             436 non-null    int64 
 1   driver_name      436 non-null    object
 2   car              436 non-null    object
 3   qualifying_time  436 non-null    int64 
 4   season           436 non-null    int64 
 5   round            436 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 23.8+ KB
None
--------------------------------------------------------------------------------
Row Count :
grid               436
driver_name        436
car                436
qualifying_time    436
season             436
round    

Build required data structures - Weather DF

In [11]:
start_time = datetime.datetime.now()
logStep("Build required data structures - Weather DF")
weather = races.iloc[:,[0,1,2]]
info    = []
for link in races.url:
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()
                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima)  
    except Exception as e:
        info.append('not found')
weather['weather'] = info
weather_dict = {
    'weather_warm' : ['soleggiato','clear','warm','hot','sunny','fine','mild','sereno'],
    'weather_cold' : ['cold','fresh','chilly','cool'],
    'weather_dry'  : ['dry','asciutto'],
    'weather_wet'  : ['showers','wet','rain','pioggia','damp','thunderstorms','rainy'],  
    'weather_cloudy': ['overcast','nuvoloso','clouds','cloudy','grey','coperto']}
weather_df = pd.DataFrame(columns = weather_dict.keys())
for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)
weather_info = pd.concat([weather, weather_df], axis = 1)
printDFinfo('weather_info',weather_info)
logStep("DONE");
end_time            = datetime.datetime.now()
step07_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step07_elapsed_time} seconds")

2023-10-11 14:25:26.380592 Build required data structures - Weather DF----------
--------------------------------------------------------------------------------
Name:  weather_info
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   season          22 non-null     int64 
 1   round           22 non-null     int64 
 2   circuit_id      22 non-null     object
 3   weather         22 non-null     object
 4   weather_warm    22 non-null     int64 
 5   weather_cold    22 non-null     int64 
 6   weather_dry     22 non-null     int64 
 7   weather_wet     22 non-null     int64 
 8   weather_cloudy  22 non-null     int64 
dtypes: int64(7), object(2)
memory usage: 1.7+ KB
None
--------------------------------------------------------------------------------
Row Count :
sea

Merge all DFs into one single DF to be used by ML training and prediction

In [12]:
start_time = datetime.datetime.now()
logStep("Build required data structures - Merge DFs")
df1      = pd.merge(races, weather,               how='inner', on=['season', 'round', 'circuit_id']).drop(['lat', 'long','country','weather'],axis = 1)
df2      = pd.merge(df1,   results,               how='inner', on=['season', 'round', 'circuit_id']).drop(['points', 'status', 'time'],axis = 1)
df3      = pd.merge(df2,   driver_standings,      how='left',  on=['season', 'round', 'driver'])
df4      = pd.merge(df3,   constructor_standings, how='left',  on=['season', 'round', 'constructor'])
final_df = pd.merge(df4,   qualifying_results,    how='inner', on=['season', 'round', 'grid']).drop(['driver_name', 'car'],axis = 1)
final_df['date']          = pd.to_datetime(final_df.date)
final_df['date_of_birth'] = pd.to_datetime(final_df.date_of_birth)
final_df['driver_age']    = final_df.apply(lambda x: relativedelta(x['date'], x['date_of_birth']).years, axis=1)
final_df.drop(['date', 'date_of_birth'], axis = 1, inplace = True)
for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 'constructor_wins' , 'constructor_standings_pos']:
    final_df[col].fillna(0, inplace = True)
    final_df[col] = final_df[col].map(lambda x: int(x))
final_df.dropna(inplace = True )
final_df = final_df[final_df['qualifying_time'] != 0]
final_df.sort_values(['season', 'round', 'grid'], inplace = True)
final_df['qualifying_time_diff'] = final_df.groupby(['season', 'round']).qualifying_time.diff()
final_df['qualifying_time'] = final_df.groupby(['season','round']).qualifying_time_diff.cumsum().fillna(0)
final_df.drop('qualifying_time_diff', axis = 1, inplace = True)
final_df.reset_index(inplace = True, drop = True)
printDFinfo('final_df',final_df)
logStep("DONE");
end_time            = datetime.datetime.now()
step08_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step08_elapsed_time} seconds")

2023-10-11 14:25:36.540936 Build required data structures - Merge DFs-----------
--------------------------------------------------------------------------------
Name:  final_df
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   season                     428 non-null    int64  
 1   round                      428 non-null    int64  
 2   circuit_id                 428 non-null    object 
 3   url                        428 non-null    object 
 4   driver                     428 non-null    object 
 5   nationality                428 non-null    object 
 6   constructor                428 non-null    object 
 7   grid                       428 non-null    int64  
 8   podium                     428 non-null    int64  
 9   driver_points  

Prepare score functions - Regression and Classification

In [13]:
start_time = datetime.datetime.now()
logStep("Prepare score functions - Regression and Classification")
scaler    = StandardScaler()

def score_regression(model):
    score = 0
    for circuit in df['round'].unique():
        test = df[(df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        prediction_df = pd.DataFrame(model.predict(X_test), columns = ['results'])
        prediction_df['podium'] = y_test.reset_index(drop = True)
        prediction_df['actual'] = prediction_df.podium.map(lambda x: 1 if x == 1 else 0)
        prediction_df.sort_values('results', ascending = True, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted']        = prediction_df.index
        prediction_df['predicted']        = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)
        score += precision_score(prediction_df.actual, prediction_df.predicted)
    model_score = score / df['round'].unique().max()
    return model_score, prediction_df

def score_classification(model):
    score = 0
    for circuit in df['round'].unique():
        test = df[(df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop = True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)
        score += precision_score(prediction_df.actual, prediction_df.predicted)
    model_score = score / df['round'].unique().max()
    return model_score, prediction_df
    
logStep("DONE");
end_time            = datetime.datetime.now()
step09_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step09_elapsed_time} seconds")

2023-10-11 14:25:42.788392 Prepare score functions - Regression and Classification
2023-10-11 14:25:42.790791 DONE-------------------------------------------------
2023-10-11 14:25:42.791868 ELAPSED TIME: 0:00:00.003390 seconds-----------------


Scaling

In [14]:
start_time = datetime.datetime.now()
logStep("Scaling and Splitting")
df        = final_df.copy()
df.reset_index(inplace = True, drop = True)
df        = df.drop(['circuit_id'] , axis = 1)
df        = df.drop(['url']        , axis = 1)
df        = df.drop(['nationality'], axis = 1)
df        = df.drop(['constructor'], axis = 1)
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)
train     = df[df.season <2023]
X_train   = train.drop(['driver', 'podium'], axis = 1)
y_train   = train.podium
X_train   = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
logStep("DONE");
end_time            = datetime.datetime.now()
step10_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step10_elapsed_time} seconds")

2023-10-11 14:25:45.587503 Scaling and Splitting--------------------------------
2023-10-11 14:25:45.594372 DONE-------------------------------------------------
2023-10-11 14:25:45.594741 ELAPSED TIME: 0:00:00.007230 seconds-----------------


Linear Regression

In [15]:
start_time = datetime.datetime.now()
logStep("Linear Regression")
comparison_dict = {'model': [], 'params': [], 'score': []}
params={'fit_intercept': [True, False]}
for fit_intercept in params['fit_intercept']:
    model_params = (fit_intercept)
    model = LinearRegression(fit_intercept = fit_intercept)
    model.fit(X_train, y_train)   
    model_score, model_prediction = score_regression(model)
    comparison_dict['model'].append('linear_regression')
    comparison_dict['params'].append(model_params)
    comparison_dict['score'].append(model_score)
print(pd.DataFrame(comparison_dict).groupby('model')['score'].max())
predictions  = model.predict(X_train)
target_names = df.driver.unique()
newpred      = np.zeros(len(predictions))
for i in range(len(predictions)):
  newpred[i] = predictions[i]
p = np.nan_to_num(newpred)
j = 0
for i in p:
    p[j] = int(i*100)
    j    = j + 1
y  = np.nan_to_num(y_train) 
print(classification_report(y, p, target_names = target_names,labels=np.unique(p)))
logStep("DONE");
end_time            = datetime.datetime.now()
step11_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step11_elapsed_time} seconds")

2023-10-11 14:25:49.877228 Linear Regression------------------------------------
model
linear_regression    0.0
Name: score, dtype: float64
                 precision    recall  f1-score   support

        leclerc       0.00      0.00      0.00         0
 max_verstappen       0.00      0.00      0.00         0
          sainz       0.00      0.00      0.00         0
          perez       0.00      0.00      0.00         0
       hamilton       0.00      0.00      0.00         0
         bottas       0.00      0.00      0.00         0
kevin_magnussen       0.00      0.00      0.00         0
         alonso       0.00      0.00      0.00         0
        russell       0.00      0.00      0.00         0
          gasly       0.00      0.00      0.00         0
           ocon       0.00      0.00      0.00         0
mick_schumacher       0.00      0.00      0.00         0
         norris       0.00      0.00      0.00         0
          albon       0.98      0.13      0.23       406
    

Random Forest Regressor

In [16]:
start_time = datetime.datetime.now()
logStep("Random Forest Regressor")
params={'criterion': ['friedman_mse'],
        'max_features': [0.8, 1, None],
        'max_depth': [None]}
for criterion in params['criterion']:
    for max_features in params['max_features']:
        for max_depth in params['max_depth']:
            model_params = (criterion, max_features, max_depth)
            model = RandomForestRegressor(criterion    = criterion,
                                          max_features = max_features, 
                                          max_depth    = max_depth, 
                                          random_state = 1)
            model.fit(X_train, y_train)
            model_score, prediction_df = score_regression(model)
            comparison_dict['model'].append('random_forest_regressor')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)
print(pd.DataFrame(comparison_dict).groupby('model')['score'].max())
predictions  = model.predict(X_train)
target_names = df.driver.unique()
newpred      = np.zeros(len(predictions))
for i in range(len(predictions)):
    newpred[i] = predictions[i]
p = np.nan_to_num(newpred)
j = 0
for i in p:
    p[j] = int(i*100)
    j    = j + 1
y  = np.nan_to_num(y_train) 
print(classification_report(y, p, target_names = target_names,labels=np.unique(p)))
logStep("DONE");
end_time            = datetime.datetime.now()
step12_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step12_elapsed_time} seconds")

2023-10-11 14:25:53.087475 Random Forest Regressor------------------------------
model
linear_regression          0.0
random_forest_regressor    0.0
Name: score, dtype: float64
                 precision    recall  f1-score   support

        leclerc       1.00      0.85      0.92       406
 max_verstappen       0.00      0.00      0.00        22
          sainz       0.00      0.00      0.00         0
          perez       0.00      0.00      0.00         0
       hamilton       0.00      0.00      0.00         0
         bottas       0.00      0.00      0.00         0
kevin_magnussen       0.00      0.00      0.00         0
         alonso       0.00      0.00      0.00         0
        russell       0.00      0.00      0.00         0
          gasly       0.00      0.00      0.00         0
           ocon       0.00      0.00      0.00         0
mick_schumacher       0.00      0.00      0.00         0
         norris       0.00      0.00      0.00         0
          albon       0.

Logistic Regression

In [17]:
start_time = datetime.datetime.now()
logStep("Logistic Regression")
params={'penalty': ['l1', 'l2'],
        'solver' : ['saga', 'liblinear'],
        'C'      : np.logspace(-3,1,20)}
for penalty in params['penalty']:
    for solver in params['solver']:
        for c in params['C']:
            model_params = (penalty, solver, c)
            model = LogisticRegression(penalty = penalty, solver = solver, C = c, max_iter = 10000)
            model.fit(X_train, y_train)
            model_score, model_prediction = score_classification(model)
            comparison_dict['model'].append('logistic_regression')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)
print(pd.DataFrame(comparison_dict).groupby('model')['score'].max())
predictions  = model.predict(X_train)
target_names = df.driver.unique()
newpred      = np.zeros(len(predictions))
for i in range(len(predictions)):
    newpred[i] = predictions[i]
p  = np.nan_to_num(newpred)
j  = 0
for i in p:
    p[j] = int(i*100)
    j    = j + 1
y  = np.nan_to_num(y_train) 
print(classification_report(y, p, target_names = target_names,labels=np.unique(p)))
logStep("DONE");
end_time            = datetime.datetime.now()
step13_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step13_elapsed_time} seconds")

2023-10-11 14:25:56.800928 Logistic Regression----------------------------------
model
linear_regression          0.000000
logistic_regression        0.636364
random_forest_regressor    0.000000
Name: score, dtype: float64
                 precision    recall  f1-score   support

        leclerc       0.97      0.99      0.98       406
 max_verstappen       0.00      0.00      0.00         0

      micro avg       0.94      0.99      0.97       406
      macro avg       0.48      0.50      0.49       406
   weighted avg       0.97      0.99      0.98       406

2023-10-11 14:26:00.158737 DONE-------------------------------------------------
2023-10-11 14:26:00.159035 ELAPSED TIME: 0:00:03.358106 seconds-----------------


Random Forest Classifier

In [18]:
start_time = datetime.datetime.now()
logStep("Random Forest Classifier")
params = {
    'criterion'   : ['gini', 'entropy'],
    'max_features': [None],
    'max_depth'   : [None]
    }
for criterion in params['criterion']:
    for max_features in params['max_features']:
        for max_depth in params['max_depth']:
            model_params = (criterion, max_features, max_depth)
            model = RandomForestClassifier(criterion = criterion, max_features = max_features, max_depth = max_depth)
            model.fit(X_train, y_train)
            model_score, model_prediction = score_classification(model)
            comparison_dict['model'].append('random_forest_classifier')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)
print(pd.DataFrame(comparison_dict).groupby('model')['score'].max())
predictions  = model.predict(X_train)
target_names = df.driver.unique()
newpred = np.zeros(len(predictions))
for i in range(len(predictions)):
    newpred[i] = predictions[i]
p = np.nan_to_num(newpred)
j = 0
for i in p:
    p[j] = int(i*100)
    j    = j + 1
y  = np.nan_to_num(y_train) 
print(classification_report(y, p, target_names = target_names,labels=np.unique(p)))
logStep("DONE");
end_time            = datetime.datetime.now()
step14_elapsed_time = end_time - start_time
logStep(F"ELAPSED TIME: {step14_elapsed_time} seconds")

2023-10-11 14:26:02.249640 Random Forest Classifier-----------------------------
model
linear_regression           0.000000
logistic_regression         0.636364
random_forest_classifier    1.000000
random_forest_regressor     0.000000
Name: score, dtype: float64
                 precision    recall  f1-score   support

        leclerc       1.00      1.00      1.00       406
 max_verstappen       0.00      0.00      0.00         0

      micro avg       0.95      1.00      0.97       406
      macro avg       0.50      0.50      0.50       406
   weighted avg       1.00      1.00      1.00       406

2023-10-11 14:26:02.540943 DONE-------------------------------------------------
2023-10-11 14:26:02.541270 ELAPSED TIME: 0:00:00.291626 seconds-----------------


Neural Network

In [19]:
start_time = datetime.datetime.now()
logStep("Neural Network Classifier")
params={'hidden_layer_sizes': [(60,20,40,5), (50,25,50,10)], 
        'activation'        : ['tanh', 'relu'], 
        'solver'            : 'adam', 
        'alpha'             : np.logspace(-4,2,20)} 
for hidden_layer_sizes in params['hidden_layer_sizes']:
    for activation in params['activation']:
        for alpha in params['alpha']:
            model_params = (hidden_layer_sizes, activation, 'adam', alpha )
            model = MLPClassifier(hidden_layer_sizes = hidden_layer_sizes, activation = activation, solver = 'adam', alpha = alpha, random_state = 1)
            model.fit(X_train, y_train)
            model_score, model_prediction = score_classification(model)
            comparison_dict['model'].append('neural_network_classifier')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score) 
print(pd.DataFrame(comparison_dict).groupby('model')['score'].max())
predictions = model.predict(X_train)
target_names = df.driver.unique()
newpred = np.zeros(len(predictions))
for i in range(len(predictions)):
    newpred[i] = predictions[i]
p = np.nan_to_num(newpred)
j = 0
for i in p:
    p[j] = int(i*100)
    j    = j + 1
y  = np.nan_to_num(y_train) 
print(classification_report(y, p, target_names = target_names,labels=np.unique(p)))
logStep("DONE");
end_time            = datetime.datetime.now()
step15_elapsed_time = end_time - start_time
end_time_tt         = datetime.datetime.now()
steptt_elapsed_time = end_time_tt - start_time00
logStep(F"ELAPSED TIME: {step15_elapsed_time} seconds")
logStep(F"TOT ELA TIME: {steptt_elapsed_time} seconds")

2023-10-11 14:26:06.324002 Neural Network Classifier----------------------------
model
linear_regression            0.000000
logistic_regression          0.636364
neural_network_classifier    0.772727
random_forest_classifier     1.000000
random_forest_regressor      0.000000
Name: score, dtype: float64
                 precision    recall  f1-score   support

        leclerc       0.95      1.00      0.97       406

      micro avg       0.95      1.00      0.97       406
      macro avg       0.95      1.00      0.97       406
   weighted avg       0.95      1.00      0.97       406

2023-10-11 14:26:23.675141 DONE-------------------------------------------------
2023-10-11 14:26:23.675832 ELAPSED TIME: 0:00:17.351751 seconds-----------------
2023-10-11 14:26:23.676481 TOT ELA TIME: 0:03:42.208143 seconds-----------------
